In [ ]:
import numpy as np
# create my own decision Tree
# first we need to cree the Node class
# Node is where the question is ask and the split happen if the node is not the leaf node

class Node:
    def __init__(self, features = None, threshold = None, left = None, right = None, value = None):
        self.features = features
        self.threshold = threshold
        self.left =  left
        self.right = right
        self.value = value

        # we need to check if the value is none by checking the value.
        # if the value is None which mean it is note a leaf so we can continue
        # if it is not None that mean it is  a leaf  so we stop
    def is_leaf(self):
        return self.value is not None

# Now we will create the Tree class
class DecisionTree:
    # Max_deth is the level of the Tree , How many time the tree need to be splited
    # min_sample_split is the minimun number in which the split need to stop
    # for a dataset of<100 rows we use 3-5 max_depth and  2- 5 Min_sample
    # for a dataset 100-1,000 rows we use 5-10 max_depht and 5-10 min_sample
    # for a dataset above 10,000 we use 10-20 max_deth and 10-50 min-sample according to some reaserch

    def __init__(self, max_depth = 3, min_samples_split =2):
        self.max_depth = max_depth
        self.min_sample_split = min_samples_split
        self.root =None

    # now we create a sum of squared function which help calculate the precentage of error for each try
    # we use sse for regression problem
    def _sse(self, y):
        mean = np.mean(y)
        return np.sum((y-mean)**2)

    ''' Now we need to build a function that go over all the index in feature  to try nall posible question calculate
    the percentage of error and at the and record the pass which has the minimum percentage of error
    '''
    '''def _best_split(self, X, y):
        n_samples, n_features = X.shape
        best_feature =None
        best_threshold = None
        best_score = float("inf")'''

    ''' We need to run a for loop which we go over the n_features take every row compare to threshold
        compute the ss and say it it need to go left or right '''
    '''for j in range(n_features):
            threshold = np.unique(X[:, j])
            for t in threshold:
                left_split = X[:, j] <= t # for every row j in n_feature,  if  row j <= t set True and go left  else false and go right
                right_split = ~left_split
                # for safety
                if left_split.sum() == 0 or right_split.sum() == 0:
                    continue

                # Extract y for left and right
                y_left = y[left_split]
                y_right = y[right_split]

                # compute the sse the see the score
                score = self._sse(y_left) + self._sse(y_right)

                # compare the score for every pass
                if score <best_score:
                    best_score =  score
                    best_feature = j
                    best_threshold = t
        return  best_score, best_threshold, best_feature
        '''

    def _best_split(self, X, y):
        n_samples, n_features = X.shape
        best_feature, best_threshold = None, None
        best_score = float("inf")

        for j in range(n_features):
            thresholds = np.unique(X[:, j])
            for t in thresholds:
                left_mask = X[:, j] <= t
                right_mask = ~left_mask

            if left_mask.sum() == 0 or right_mask.sum() == 0:
                continue

            y_left = y[left_mask]
            y_right = y[right_mask]

            score = self._sse(y_left) + self._sse(y_right)

            if score < best_score:
                best_score = score
                best_feature = j
                best_threshold = t

        return best_feature, best_threshold, best_score





    ''' NOW we need to buil a recursive function.  A recursive function is a function that called itsel
       we need because in every split if the node is not a leaf we need to redo the same step we did on _best_split
       so we use the recursive function so the function will call itself and we don't need to rewrite this function for node we need to split
       '''
    def _build( self, X, y, depth):
        # create the first occasion we our loop need to stop
        # this is the base case
        if len(y) < self.min_sample_split:
            leaf_value = float(np.mean(y))
            return Node(value =leaf_value)

        # base case 2
        if depth >= self.max_depth:
            leaf_value = float(np.mean(Y))
            return Node(value = leaf_value)

        # try to find the best split
        feature, threshold, score = self._best_split(X, y)

        # Now if there is no Good split found
        if feature is None:
            leaf_value = float(np.mean(y))
            return Node(value = leaf_value)

        # etract the value
        left_mask = X[:, feature] <= threshold
        right_mask = ~ left_mask

        # for safety
        if left_mask.sum() ==0 and right_mask.sum() == 0:
            leaf_value = float(np.mean(y))
            return Node (value = leaf_value)

        # Now the recursion
        # Left branch split

        left_branch = self._build(X[left_mask], y[left_mask], depth + 1)

        # right branch split
        right_branch = self._build(X[right_mask], y[right_mask], depth + 1)

        return Node (
            features = feature,
            threshold = threshold,
            left = left_branch,
            right = right_branch
        )

    # fit function to train the model
    def fit(self, X, y):
        X = np.array(X, dtype = float)
        y = np.array(y, dtype = float)
        self.root = self._build(X, y, depth= 0)


    def _predict_one( self, X, node):
        if node.is_leaf():
            return node.value

        if X[node.feature] <= node.threshold:
            return self._predict_one (X, node.left)
        else:
            return self._predict_one(X, node.right )



    # Now prediction
    def predict( self, X):
        X = np.array(X, dtype = float)
        return np.array([self._predict_one(x, self.root) for x in X])

### tis is the text code

In [ ]:
if __name__ == "__main__":
    # Simple 1D data: X is like [[1], [2], [3], ...]
    X = np.array([
        [1],
        [2],
        [3],
        [4],
        [5]
    ])
    # y is some numbers we want to predict
    y = np.array([1.2, 1.9, 3.1, 3.9, 5.2])

    # Create the tree with some settings
    tree = DecisionTree(max_depth=2, min_samples_split=2)

    # STEP 3.1 : Train the tree
    tree.fit(X, y)

    # STEP 3.2 : Predict on the same X (just to see it works)
    preds = tree.predict(X)

    print("X:\n", X.ravel())
    print("True y:", y)
    print("Pred y:", preds)

X:
 [1 2 3 4 5]
True y: [1.2 1.9 3.1 3.9 5.2]
Pred y: [3.06 3.06 3.06 3.06 3.06]
